In [ ]:
# Extract results

In [ ]:
### Imports 
import os
import math
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.patches as patches
from matplotlib.collections import LineCollection
import time
import json

import pandas as pd

sys.path.append('../../')
from script.conversion.bison.coordinates import rd_to_utm
from mnms.graph.layers import PublicTransportLayer, MultiLayerGraph, OriginDestinationLayer, SharedVehicleLayer
from mnms.generation.roads import generate_pt_line_road, generate_one_zone
from mnms.generation.layers import generate_layer_from_roads
from mnms.vehicles.veh_type import Tram, Metro, Bus, Bike
from mnms.generation.zones import generate_one_zone
from mnms.mobility_service.public_transport import PublicTransportMobilityService
from mnms.mobility_service.vehicle_sharing import VehicleSharingMobilityService
from mnms.time import TimeTable, Dt, Time
from mnms.io.graph import load_graph, save_graph, load_odlayer, save_transit_links
from mnms.tools.render import draw_roads, draw_line, draw_odlayer, draw_path, draw_veh_activity
#from mnms.tools.geometry import points_in_polygon, get_bounding_box
from mnms.time import Time

In [ ]:
### Parameters

# Files and directories
f = open('params.json')
params = json.load(f)

current_dir = os.getcwd()
indir = current_dir + '/inputs/'
outdir = current_dir + '/outputs/'

#coord_csv_filepath = indir + 'KV1_GVB_2609_2/Csv/POINT.csv' # file with coordinates of the network
#amsterdam_json_filepath = indir + 'new_network.json' # mlgraph with the road network only
#amsterdam_json_filepath_pt_transit = indir + "network_pt_transit.json"
#transit_path = indir + "transit.json"

FIG_SIZE_HALF = (6,3)
FONT_SIZE_LAB = 14
FONT_SIZE_LEG = 12
FONT_SIZE_AXI = 12

In [ ]:
### Load network

mmgraph_pt = load_graph(indir + params["fn_network"])
df_stations = pd.read_csv(indir+params['fn_emoped_st_init'])

#df_emoped1 = pd.read_csv(indir + 'init_pos_emoped.csv')
#df_emoped2 = pd.read_csv(indir + 'init_pos_emoped.csv')

### Load demand

df_dmd = pd.read_csv(indir + params['fn_demand'], sep=";")

### Load odlayer
odlayer = load_odlayer(indir + params["fn_odlayer"])
x_od = []
y_od = []
for i in odlayer.origins.items():
    x = i[1][0]
    y = i[1][1]
    x_od.append(x)
    y_od.append(y)

In [ ]:
## Load results

df_emoped1_notax = pd.read_csv(outdir+'notax/emoped1_veh.csv', sep=";")
df_path_notax = pd.read_csv(outdir+'notax/path.csv', sep=";")
df_users_notax = pd.read_csv(outdir+'notax/users.csv', sep=";")

df_emoped1_tax = pd.read_csv(outdir+'tax/emoped1_veh.csv', sep=";")
df_path_tax = pd.read_csv(outdir+'tax/path.csv', sep=";")
df_users_tax = pd.read_csv(outdir+'tax/users.csv', sep=";")

df_emoped1_subsidy = pd.read_csv(outdir+'subsidy/emoped1_veh.csv', sep=";")
df_path_subsidy = pd.read_csv(outdir+'subsidy/path.csv', sep=";")
df_users_subsidy = pd.read_csv(outdir+'subsidy/users.csv', sep=";")

df_emoped1_subsidytax = pd.read_csv(outdir+'subsidytax/emoped1_veh.csv', sep=";")
df_path_subsidytax = pd.read_csv(outdir+'subsidytax/path.csv', sep=";")
df_users_subsidytax = pd.read_csv(outdir+'subsidytax/users.csv', sep=";")

"""df_emoped1_tax = pd.read_csv(outdir+'notax/emoped1_veh.csv', sep=";")
df_path_tax = pd.read_csv(outdir+'notax/path.csv', sep=";")
df_users_tax = pd.read_csv(outdir+'notax/users.csv', sep=";")

df_emoped1_subsidy = pd.read_csv(outdir+'notax/emoped1_veh.csv', sep=";")
df_path_subsidy = pd.read_csv(outdir+'notax/path.csv', sep=";")
df_users_subsidy = pd.read_csv(outdir+'notax/users.csv', sep=";")

df_emoped1_subsidytax = pd.read_csv(outdir+'notax/emoped1_veh.csv', sep=";")
df_path_subsidytax = pd.read_csv(outdir+'notax/path.csv', sep=";")
df_users_subsidytax = pd.read_csv(outdir+'notax/users.csv', sep=";")"""

In [ ]:
# Look vehicle trajectory
"""veh_id = 130
df = df_emoped1[df_emoped1["ID"] == int(veh_id)]
list_pos_emoped = np.zeros((len(df['POSITION']),2))
for i, row in enumerate(df['POSITION']):
    pos = row.split(' ')
    #plt.text(float(pos[0]), float(pos[1]), row.TIME)
    list_pos_emoped[i,:] = [float(pos[0]), float(pos[1])]"""

# Demand

In [ ]:
len(df_dmd)

In [ ]:
df_dmd

In [ ]:
origin_points = np.asarray([[float(x) for x in s.split(' ')] for s in df_dmd.ORIGIN.values])
plt.hexbin(origin_points[:,0],origin_points[:,1], gridsize=20, cmap='Purples')

In [ ]:
destination_points = np.asarray([[float(x) for x in s.split(' ')] for s in df_dmd.DESTINATION.values])
plt.hexbin(destination_points[:,0],destination_points[:,1], gridsize=20, cmap='Purples')

In [ ]:
def calc_dist(row):
    pos1 = [float(x) for x in row.ORIGIN.split(' ')]
    pos2 = [float(x) for x in row.DESTINATION.split(' ')]
    #dist = abs(pos1[0]-pos2[0]) + abs(pos1[1]-pos2[1])
    dist = ((pos1[0]-pos2[0])**2 + (pos1[1]-pos2[1])**2)**0.5
    return dist

dist = df_dmd.apply(calc_dist, axis=1)

In [ ]:
plt.hist(dist, bins=100);

In [ ]:
# Look O/D for users wihtout paths
list_nomatch = []
list_origins = []
list_destinations = []
for i, row in df_path_notax[:].iterrows():
    if pd.isna(row['PATH']):
        list_nomatch.append(row['ID'])
        user = df_dmd[df_dmd['ID']==row['ID']]
        list_origins.append([float(o) for o in user['ORIGIN'].iloc[0].split(' ')])
        list_destinations.append([float(o) for o in user['DESTINATION'].iloc[0].split(' ')])

In [ ]:
"""id_user = '1066750-1'
o_user =  [float(o) for o in df_dmd[df_dmd['ID']==id_user]['ORIGIN'].iloc[0].split(' ')]
d_user =  [float(o) for o in df_dmd[df_dmd['ID']==id_user]['DESTINATION'].iloc[0].split(' ')]

dist_o = [(o_user[0]-x)**2 + (o_user[1]-y)**2 for (x,y) in zip(x_od,y_od)]
dist_d = [(d_user[0]-x)**2 + (d_user[1]-y)**2 for (x,y) in zip(x_od,y_od)]
print(np.argmin(dist_o), np.argmin(dist_d))"""

# Plot network

In [ ]:
polygon = np.asarray(params['polygon_demand'])

In [ ]:
x_st = []
y_st = []
for n in df_stations['closest_node']:
    x_st.append(mmgraph_pt.roads.nodes[n].position[0])
    y_st.append(mmgraph_pt.roads.nodes[n].position[1])

In [ ]:
x_nodes = []
y_nodes = []
for key in mmgraph_pt.roads.nodes.keys():
    pos = mmgraph_pt.roads.nodes[key].position
    x_nodes.append(pos[0])
    y_nodes.append(pos[1])

In [ ]:
print([p - polygon[0,0] for p in polygon[:,0]])
print([p - polygon[0,1] for p in polygon[:,1]])

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

draw_roads(ax, mmgraph_pt.roads, color='grey', linkwidth=0.1, nodesize=0, draw_stops=False, node_label=False)

# Params of the visualization
colors = {'BUS': 'green', 'METRO': 'red', 'TRAM': 'skyblue'}

for layer in mmgraph_pt.layers.values():
    if type(layer) == PublicTransportLayer:
        for name, line in layer.lines.items():
                draw_line(ax, mmgraph_pt, line, color=colors[name[:name.find('_')]], 
                          linkwidth=0.4, nodesize=1, line_label=None, label_size=1, alpha=1., stopmarkeredgewidth=0.1)

#plt.plot(x_od, y_od, 'dk', alpha=0.4)

#plt.plot(x_st, y_st, '*b')

#plt.plot(x_nodes, y_nodes, '.m', alpha=0.5, markersize=1)

#plt.plot(list_pos_emoped[:,0], list_pos_emoped[:,1], ':.')

for o,d in zip(list_origins, list_destinations):
    plt.plot([o[0], d[0]], [o[1], d[1]], 'm:', alpha=0.2)
    plt.plot(d[0], d[1], 'dm', alpha=0.2)
    plt.plot(o[0], o[1], 'om', alpha=0.2)

#plt.plot([o_user[0], d_user[0]], [o_user[1], d_user[1]], 'o-')
x=list(polygon[:,0])
x.append(polygon[0,0])
y=list(polygon[:,1])
y.append(polygon[0,1])
#plt.plot(x, y, '+--k')

legend = [Line2D([0, 1], [0, 1], marker='.', markersize=12, markeredgecolor='black', color='skyblue', linewidth=5,
            label='Tram line'),
          Line2D([0, 1], [0, 1], marker='.', markersize=12, markeredgecolor='black', color='red', linewidth=5,
            label='Metro line'),
          Line2D([0, 1], [0, 1], marker='.', markersize=12, markeredgecolor='black', color='green', linewidth=5,
            label='Bus line'),
          Line2D([0, 1], [0, 1], marker='.', markersize=0, markeredgecolor='grey', markerfacecolor='grey', color='grey', linewidth=1,
            label='Roads')]
#          Line2D([0, 1], [0, 1], marker='*', markersize=10, color='b', linewidth=0,
#            label='Virtual station')]

plt.hexbin(destination_points[:,0],destination_points[:,1], gridsize=20, cmap='Purples')
#plt.hexbin(origin_points[:,0],origin_points[:,1], gridsize=20, cmap='Greens')

legend = plt.legend(handles=legend, fontsize=FONT_SIZE_LEG)
plt.tight_layout() 
#plt.legend(, ncol=2, loc='lower left', bbox_to_anchor=(0.1,1),)
plt.xticks(fontsize=FONT_SIZE_AXI)
plt.yticks(fontsize=FONT_SIZE_AXI)
plt.xlim([617000,640000])
plt.ylim([5.793e6, 5.813e6])
#plt.savefig(params['figdir']+'network_light.pdf', bbox_inches = 'tight')


## Modes shares

In [ ]:
def emoped_in_modes(modes):
    if pd.isna(modes):
        return False
    else:
        return 'emoped1' in modes

def pt_in_modes(modes):
    if pd.isna(modes):
        return False
    else:
        return ('BUS' in modes) or ('TRAM' in modes) or ('METRO' in modes)

def calculate_mode_shares(df_paths, list_id):
    emoped_only = 0
    pt_only = 0
    combined = 0
    no_match = 0
    for id in list_id[:]:
        paths = df_paths[df_paths['ID']==id]
        path = paths.iloc[-1]
        modes = path['SERVICES']
        if emoped_in_modes(modes):
            if pt_in_modes(modes):
                combined+=1
            if (not pt_in_modes(modes)):
                emoped_only+=1
        elif pt_in_modes(modes):
            pt_only+=1
        else:
            no_match+=1
    sum = len(list_id)
    return (np.around(emoped_only/sum,3), np.around(pt_only/sum,3), np.around(combined/sum,3), np.around(no_match/sum,3))

In [ ]:
list_id = df_path_notax['ID'].unique()

print('notax', calculate_mode_shares(df_path_notax, list_id))
#print('tax', calculate_mode_shares(df_path_tax, list_id))
#print('subsidy', calculate_mode_shares(df_path_subsidy, list_id))
#print('subsidytax', calculate_mode_shares(df_path_subsidytax, list_id))

In [ ]:
df_path_notax.CHOSEN.unique()

In [ ]:
df_path = df_path_notax

list_ser=[]
pt_before = []
pt_after = []
pt_before_after = []
for service, i_user in zip(df_path.SERVICES, df_path.ID):
    if pd.isna(service): 
        service = ''
    ser = service.replace('WALK','')
    ser = ser.replace(' ','')
    if emoped_in_modes(ser) and pt_in_modes(ser):
        a = ser.split('emoped1')
        if len(a[0])>0 and len(a[-1])>0:
            pt_before_after.append(i_user)
        elif len(a[0])>0:
            pt_before.append(i_user)
        elif len(a[-1])>0:
            pt_after.append(i_user)

In [ ]:
o_pt_before = []
d_pt_before = []
df = df_dmd.set_index('ID')
for i_user in pt_before:
    user = df.loc[i_user]
    o_pt_before.append([float(o) for o in user['ORIGIN'].split(' ')])
    d_pt_before.append([float(o) for o in user['DESTINATION'].split(' ')])
o_pt_before = np.asarray(o_pt_before)
d_pt_before = np.asarray(d_pt_before)

o_pt_after = []
d_pt_after = []
for i_user in pt_after:
    user = df.loc[i_user]
    o_pt_after.append([float(o) for o in user['ORIGIN'].split(' ')])
    d_pt_after.append([float(o) for o in user['DESTINATION'].split(' ')])
o_pt_after = np.asarray(o_pt_after)
d_pt_after = np.asarray(d_pt_after)

In [ ]:
print(pt_before, pt_after, pt_before_after)

## TTT/TTD

In [ ]:
def calculate_tt_td(list_id, df_users):
    tt = []
    td = []
    #count = 0
    for id in list_id:
        df = df_users[df_users['ID']==id]
        if len(df)>1:
            tt.append(Time(df['TIME'].iloc[-1]).to_seconds() - Time(df['TIME'].iloc[0]).to_seconds())
            td.append(df['DISTANCE'].iloc[-1])
            #count += 1
    return (tt, td)

In [ ]:
# Compute total travel time/distance

tt_notax, td_notax = calculate_tt_td(list_id, df_users_notax)
tt_tax, td_tax = calculate_tt_td(list_id, df_users_tax)
tt_subsidy, td_subsidy = calculate_tt_td(list_id, df_users_subsidy)
tt_subsidytax, td_subsidytax = calculate_tt_td(list_id, df_users_subsidytax)

In [ ]:
plt.hist(tt_notax, alpha=0.5, label = "notax")
plt.hist(tt_tax, alpha=0.5, label = "tax")
plt.hist(tt_subsidy, alpha=0.5, label = "subsidy")
plt.hist(tt_subsidytax, alpha=0.5, label = "subsidytax")
plt.legend()

## Emoped usage

In [ ]:
def calculate_emoped_indicators(df_emoped):
    list_emoped = df_emoped['ID'].unique()
    distances = []
    TTD = 0
    TTT = 0
    nb_rides = 0
    for id in list_emoped:
        df = df_emoped[df_emoped['ID']==id]
        TTD += df['DISTANCE'].iloc[-1]
        
        nb_rides_loc = sum(df.STATE=='STOP')-1
        #nb_rides += nb_rides_loc
        i_stops = np.where(df.STATE=='STOP')[0]
        for i in range(nb_rides_loc):
            i_start = i_stops[i]
            i_start_tt = i_stops[i]+1
            i_stop = i_stops[i+1]
            if i_start+1==i_stop:
                pass 
                #print(id)
            else:
                TTT += str_to_time(df.TIME.iloc[i_stop]) - str_to_time(df.TIME.iloc[i_start_tt])
                distances.append(df.DISTANCE.iloc[i_stop] - df.DISTANCE.iloc[i_start])
                nb_rides += 1
                if (df.DISTANCE.iloc[i_stop] - df.DISTANCE.iloc[i_start])<100:
                    print(id, i_stops)
    return (TTD*1e-3, TTT/3600, nb_rides, distances)

def str_to_time(time_str):
    return sum([float(t)*60**(2-i) for i,t in enumerate(time_str.split(':'))])

In [ ]:
ttd_notax, ttt_notax, nb_rides_notax, dist_notax = calculate_emoped_indicators(df_emoped1_notax)
ttd_tax, ttt_tax, nb_rides_tax, dist_tax = calculate_emoped_indicators(df_emoped1_tax)
ttd_subsidy, ttt_subsidy, nb_rides_subsidy, dist_subsidy = calculate_emoped_indicators(df_emoped1_subsidy)
ttd_subsidytax, ttt_subsidytax, nb_rides_subsidytax, dist_subsidytax = calculate_emoped_indicators(df_emoped1_subsidytax)

In [ ]:
print(sum(np.asarray(dist)==0))
print(ttd_notax/ttt_notax, 'km/h')

In [ ]:
rev_notax = 1*nb_rides_notax + 0.33*ttt_notax*60
rev_tax = 1*nb_rides_tax + 0.33*ttt_tax*60
rev_subsidy = 1*nb_rides_subsidy + 0.33*ttt_subsidy*60
rev_subsidytax = 1*nb_rides_subsidytax + 0.33*ttt_subsidytax*60

In [ ]:
print(ttd_notax, ttt_notax, nb_rides_notax, rev_notax)
print(ttd_tax, ttt_tax, nb_rides_tax, rev_tax)
print(ttd_subsidy, ttt_subsidy, nb_rides_subsidy, rev_subsidy)
print(ttd_subsidytax, ttt_subsidytax, nb_rides_subsidytax, rev_subsidytax)

In [ ]:
# Calibration
df_dist_mhtn = pd.read_csv('inputs/processed_felyx_data/df_dist_mhtn.csv')
df_dist_eucl = pd.read_csv('inputs/processed_felyx_data/df_dist_eucl.csv')

plt.hist(np.asarray(dist_notax)*1e-3, bins=100, cumulative=True, density=True);
plt.plot(df_dist_mhtn.dist*1e-3, df_dist_mhtn.dsty, label='Manhattan')
plt.plot(df_dist_eucl.dist*1e-3, df_dist_eucl.dsty, label='Euclidian')
plt.legend(fontsize=FONT_SIZE_LEG)
plt.xlim([0,15])
plt.tight_layout() 
#plt.legend(, ncol=2, loc='lower left', bbox_to_anchor=(0.1,1),)
plt.xticks(fontsize=FONT_SIZE_AXI)
plt.yticks(fontsize=FONT_SIZE_AXI)
plt.xlabel('Distance (km)', fontsize=FONT_SIZE_LAB)
plt.ylabel('Cumulative density', fontsize=FONT_SIZE_LAB)
#plt.savefig(params['figdir']+'dist_distribution.pdf', bbox_inches = 'tight')

In [ ]:
df_emoped1_notax

In [ ]:
list_emoped = df_emoped1_notax['ID'].unique()
d = []
pos = []
for id in list_emoped:
    df = df_emoped1_notax[df_emoped1_notax['ID']==id]
    d.append(df['DISTANCE'].iloc[-1])
    if df['DISTANCE'].iloc[-1]==0:
        pos.append([float(x) for x in df.POSITION.iloc[-1].split(' ')])
plt.hist(d, bins=50);
pos = np.array(pos)

In [ ]:
sum([d_==0 for d_ in d])

In [ ]:
position_stations = []
for key in mmgraph_pt.roads.stops.keys():
    position_stations.append(mmgraph_pt.roads.stops[key].absolute_position)

dist_o = []
for o in origin_points:
    distances = [(o[0]-s[0])**2+(o[1]-s[1])**2 for s in position_stations]
    dist_o.append(np.sqrt(min(distances)))

dist_d = []
for d in destination_points:
    distances = [(d[0]-s[0])**2+(d[1]-s[1])**2 for s in position_stations]
    dist_d.append(np.sqrt(min(distances)))

mask_o = [d>500 for d in dist_o]
mask_d = [d>500 for d in dist_d]

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

draw_roads(ax, mmgraph_pt.roads, color='grey', linkwidth=0.1, nodesize=0, draw_stops=False, node_label=False)

# Params of the visualization
colors = {'BUS': 'green', 'METRO': 'red', 'TRAM': 'skyblue'}

for layer in mmgraph_pt.layers.values():
    if type(layer) == PublicTransportLayer:
        for name, line in layer.lines.items():
                draw_line(ax, mmgraph_pt, line, color=colors[name[:name.find('_')]], 
                          linkwidth=0.4, nodesize=1, line_label=None, label_size=1, alpha=1., stopmarkeredgewidth=0.1)

#plt.plot(x_od, y_od, 'dk', alpha=0.4)

#plt.plot(x_st, y_st, '*b')

#plt.plot(x_nodes, y_nodes, '.m', alpha=0.5, markersize=1)

#plt.plot(list_pos_emoped[:,0], list_pos_emoped[:,1], ':.')

plt.hexbin(pos[:,0],pos[:,1], gridsize=20, cmap='Greens')

#plt.plot(pos[:,0],pos[:,1], '+')
#plt.plot(pos[:,0],pos[:,1]+600, 'x')

#plt.hexbin(o_pt_before[:,0],o_pt_before[:,1], gridsize=20, cmap='Purples')
#plt.hexbin(d_pt_before[:,0],d_pt_before[:,1], gridsize=20, cmap='Purples')
#plt.hexbin(o_pt_after[:,0],o_pt_after[:,1], gridsize=20, cmap='Purples')
#plt.hexbin(d_pt_after[:,0],d_pt_after[:,1], gridsize=20, cmap='Purples')

x=list(polygon[:,0])
x.append(polygon[0,0])
y=list(polygon[:,1])
y.append(polygon[0,1])
#plt.plot(x, y, '+--k')

plt.plot(origin_points[mask_o,0],origin_points[mask_o,1], 'b+')
plt.plot(destination_points[mask_o,0],destination_points[mask_o,1], 'bx')

plt.plot(origin_points[mask_d,0],origin_points[mask_d,1], 'm+')
plt.plot(destination_points[mask_d,0],destination_points[mask_d,1], 'mx')


## Station evolution

In [ ]:
df_stations['position'] = df_stations.apply(lambda row: '%.3f %.3f'%(row.x_node, row.y_node), axis=1)

In [ ]:
df_stations.position

In [ ]:
station_emoped = np.zeros((len(df_stations), df_stations.nb_emoped.sum()))
nb_emoped_stations = np.zeros((len(df_emoped1_notax.TIME.unique()), len(df_stations)), int)
for i_t, t in enumerate(df_emoped1_notax.TIME.unique()):
    df = df_emoped1_notax[df_emoped1_notax.TIME==t]
    df2 = df[df.STATE=='STOP']
    for i,row in df2.iterrows():
        i_sta = df_stations.index[df_stations.position==row.POSITION][0]
        station_emoped[:, row.ID] = 0
        station_emoped[i_sta, row.ID] = 1
    df3 = df[df.STATE!='STOP']
    for i,row in df3.iterrows():
        station_emoped[:, row.ID] = 0
    nb_emoped_stations[i_t] = station_emoped.sum(axis=1)
    

In [ ]:
df_stations.index[df_stations.position==row.POSITION][0]

In [ ]:
plt.plot(nb_emoped_stations);
#plt.ylim([0,5])

In [ ]:
station_id = []
for pos_ in pos:
    station = df_stations.index[df_stations.position == '%.3f %.3f'%(pos_[0], pos_[1])]
    station_id.append(station)

In [ ]:
station_id_unique = np.unique(station_id)

In [ ]:
plt.plot(nb_emoped_stations[:,station_id_unique]);

In [ ]:
station_id_unique

In [ ]:
net_length = 0
for name_sect in mmgraph_pt.roads.sections:
    sect = mmgraph_pt.roads.sections[name_sect]
    net_length += sect.length
print(net_length*1e-3)

In [ ]:
pos[0]

In [ ]:
alpha=20/3600
fee1 = 0.33/60
fee2 = 1
v_walk = 1.42
v_emoped = 7.5

print(fee2/(alpha/v_walk - (alpha+fee1)/v_emoped))